In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [2]:
train_data=pd.read_csv('/Users/wudailing/Downloads/bike-sharing-demand/train.csv')
test_data=pd.read_csv('/Users/wudailing/Downloads/bike-sharing-demand/test.csv')

In [3]:
train_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
datetime      10886 non-null object
season        10886 non-null int64
holiday       10886 non-null int64
workingday    10886 non-null int64
weather       10886 non-null int64
temp          10886 non-null float64
atemp         10886 non-null float64
humidity      10886 non-null int64
windspeed     10886 non-null float64
casual        10886 non-null int64
registered    10886 non-null int64
count         10886 non-null int64
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.6+ KB


In [5]:
train_data.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [6]:
#去掉outlier
print('the shape before cutting outlier:', train_data.shape)
train_data=train_data[np.abs(train_data['count']-train_data['count'].mean())<=(3*train_data['count'].std())]
print('the shape after cutting outlier:', train_data.shape)

the shape before cutting outlier: (10886, 12)
the shape after cutting outlier: (10739, 12)


In [7]:
#處理datetime
import datetime

train_data['date']=train_data['datetime'].apply(lambda x:x.split()[0])
train_data['hour']=train_data['datetime'].apply(lambda x:x.split()[1])
train_data['year']=train_data['date'].apply(lambda x:x.split('-')[0])
train_data['weekday']=train_data['date'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d').weekday())
train_data['month']=train_data['date'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d').month)


test_data['date']=test_data['datetime'].apply(lambda x:x.split()[0])
test_data['hour']=test_data['datetime'].apply(lambda x:x.split()[1])
test_data['year']=test_data['date'].apply(lambda x:x.split('-')[0])
test_data['weekday']=test_data['date'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d').weekday())
test_data['month']=test_data['date'].apply(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d').month)


In [8]:
#處理windspeed的0重新估計
import seaborn as sn
sn.distplot(train_data['windspeed'])


/anaconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [9]:
wind0=train_data[train_data['windspeed']==0]
windnot0=train_data[train_data['windspeed']!=0]
windcolumns=['year','month','season','weather','humidity','temp','atemp']

rfmodel_wind=RandomForestRegressor(n_estimators=1000,random_state=42)
rfmodel_wind.fit(windnot0[windcolumns],windnot0['windspeed'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [10]:
wind0newspeed=rfmodel_wind.predict(wind0[windcolumns])
wind0['windspeed']=wind0newspeed
train_data=windnot0.append(wind0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
train_data=train_data.drop(['datetime','hour','date','casual','registered'],axis=1)
test_datetime=test_data['datetime']
test_data=test_data.drop(['datetime','hour','date'],axis=1)

In [12]:
train_x=train_data.copy().drop('count',axis=1)
train_y=np.log(train_data['count'])


In [13]:
#隨機森林 預測腳踏車租借數

rfmodel=RandomForestRegressor(n_estimators=1000,random_state=42)
rfmodel.fit(train_x,train_y)
pred_y=rfmodel.predict(test_data)

In [14]:
submission = pd.DataFrame({
        "datetime": test_datetime,
        "count": [max(0, x) for x in np.exp(pred_y)]
    })
submission.to_csv('bike_predictions_RF.csv', index=False)